In [20]:
import os
import sys
import copy
import json
import time
import shutil
import time

import torch
from torch.autograd import Variable
from torch import optim
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# TODO: Fix sibling directory imports
cwd = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(cwd, '.'))

from src.datautils.sandia import SandiaDataProvider
from src.utils import show_batch_of_images
from src.models import mlp_net as mlp
from src.reasoning_agents import FFNReasoningAgent, RNN_RA
from src.classifiers import PairwiseClassifier
from src.utils import show_batch_of_images, dict_of_lists_to_list_of_dicts
from src.ra_training import train_reasoning_agent, train_iteration
from src.autoencoders import Conv2DAutoencoder, PCAAutoencoder, FeedforwardAutoencoder
from src.utiq import UTIQ

import src
from src import train_test, e2e_training
import importlib
importlib.reload(src.utils)
importlib.reload(src.train_test);
importlib.reload(src.e2e_training);
from src.train_test import train, test
from src.e2e_training import train_e2e

# Run from root

In [21]:
%cd ../src

/media/yasen/storage/studies/uoe/mlp/sem2/mlp-03/src


In [3]:
hyperparams = {
    'img_size': [28],
    'encoding_size': [50, 100, 200, 300],

    'ae_type': ['ff'],
    'ae_hidden_sizes': [[400], [400, 300], [300, 200], [200, 100]],

    'ra_type': ['ff'],
    'ra_hidden_size': [50, 100, 200],
    'ra_num_layers': [1, 2],
    'ra_nonlinearity': ['relu'],
    'ra_network_type': ['vanilla'],

    'clf_type': ['pw'],
    'clf_hidden_sizes': [[100], [200, 100], [50], [100, 50], [50, 30], [80, 50, 10]],
    'clf_nonlinearity': ['relu'],

    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'momentum': [0.1, 0.5, 0.9],
    'weight_decay': [0, 0.05],
    'num_epochs': [80],
    'epoch_patience': [15],
    'batch_size': [8, 16, 32, 64]
}

hyperparams = dict_of_lists_to_list_of_dicts(hyperparams)

In [22]:
hyperparams = {
    'img_size': [28],
    'encoding_size': [50],

    'ae_type': ['identity'],
    'ae_hidden_sizes': [[400]],
    'ae_pretrain': [False],
    'ae_freeze': [False],

    'ra_type': ['ff'],
    'ra_hidden_size': [100],
    'ra_num_layers': [2],
    'ra_nonlinearity': ['relu'],
    'ra_network_type': ['vanilla'],
    'ra_use_batchnorm': [False],

    'clf_type': ['lse'],
    'clf_hidden_sizes': [[80, 40], [100]],
    'clf_nonlinearity': ['relu'],
    'clf_use_batchnorm': [False],
    
    'learning_rate': [0.05],
    'momentum': [0.5],
    'weight_decay': [0.05],
    'num_epochs': [50],
    'epoch_patience': [10],
    'batch_size': [64]
}

hyperparams = dict_of_lists_to_list_of_dicts(hyperparams)

In [23]:
use_cuda = True
plot = True
for res in train_e2e(hyperparams, use_cuda=use_cuda, verbose=True, plot=plot):
    pass

1 out of 2:
(<class 'ValueError'>, ValueError('too many values to unpack (expected 2)',), <traceback object at 0x7f9876901cc8>)
2 out of 2:
(<class 'ValueError'>, ValueError('too many values to unpack (expected 2)',), <traceback object at 0x7f98769b14c8>)
